In [31]:
from pgmpy.models import DiscreteBayesianNetwork
from pgmpy.factors.discrete import TabularCPD

In [32]:
SampleFoodBN = DiscreteBayesianNetwork(
    [("Apple", "Feasibility"), ("Banana", "Feasibility"), ("Feasibility", "Success")]
)

In [33]:
cpd_apple = TabularCPD(
    variable="Apple", variable_card=3, values=[[1 / 4], [1 / 2], [1 / 4]] # Expired, Uncertain, Expired
)
cpd_banana = TabularCPD(
    variable="Banana", variable_card=3, values=[[1 / 3], [1 / 3], [1 / 3]]
)

In [34]:
import random
p1s = [random.random() for _ in range(9)]
p2s = [1 - p1s[i] for i in range(9)]
cpd_feasibility = TabularCPD(
    variable="Feasibility",
    variable_card=2,
    values=[p1s, p2s],
    evidence=["Apple", "Banana"],
    evidence_card=[3, 3],
)

In [35]:
cpd_success = TabularCPD(
    variable='Success',
    variable_card=2,
    values=[[1/3, 1/2], [2/3, 1/2]],
    evidence=["Feasibility"],
    evidence_card=[2]
)

In [36]:
SampleFoodBN.add_cpds(cpd_apple, cpd_banana, cpd_feasibility, cpd_success)

In [37]:
SampleFoodBN.check_model()

True

In [38]:
from pgmpy.inference import VariableElimination

infer = VariableElimination(SampleFoodBN)

In [39]:
p = infer.query(["Success"])
print(p)

+------------+----------------+
| Success    |   phi(Success) |
+============+================+
| Success(0) |         0.4225 |
+------------+----------------+
| Success(1) |         0.5775 |
+------------+----------------+


# Dynamically generate a BN for inference

Plan: given recipe, fridge (with conditions and food) $\rightarrow$ generates a BN based on inputs
Outcome: After generating BN, extract evidence from Fridge $\rightarrow$ infer using BN and get the probability of recipe

*Evidence*: Freshness

## How to set up BN?
### Frehness nodes at the top levels:
directly read from recipe's item list

### CPD of each item
CPT of freshness
1. freshness categorize into Expired, Not Expired, Close to Expired
2. Problems: How to assign probability? ans: P(F_i) = f(fridge condition, item_type)

## How to extract evidence? 
How to use information such as expiry dates to categorize freshness of items (because of the use of discrete)

## Alternative thoughts:

Extracting freshness can be another extended BN framework. 

For instance, freshness can depends on some more "obvious" attributes of food such as expiry date, food category. The category can be determined directly, but CPT is determined by some more "obvious" factors such as fridge condition, i.e. P(Category) = f(fridge condition); then category can be saved as a food attributes which will be inside fridge

**Drawback**: may need to build more subnetworks